In [1]:
from functools import reduce
import numpy as np
import networkx as nx
# Python translation of my JS implementation
# https://github.com/rruff82/LS-Ses17-PegGame/

In [2]:
BOARD_SIZE = 7
CORNER_SIZE = 2
NUM_PEGS = BOARD_SIZE**2-4*CORNER_SIZE**2
CENTER_PEG = int((NUM_PEGS-1)/2)
print(NUM_PEGS,CENTER_PEG)

33 16


In [3]:
INITIAL_STATE = [1 if x != CENTER_PEG else 0 for x in range(NUM_PEGS)]
TERMINAL_STATE = [0 if x != CENTER_PEG else 1 for x in range(NUM_PEGS)]


In [4]:
SIZE_OF_ROW = [BOARD_SIZE-2*CORNER_SIZE if \
               (x < CORNER_SIZE or x >= BOARD_SIZE-CORNER_SIZE) \
               else BOARD_SIZE for x in range(BOARD_SIZE)]
print(SIZE_OF_ROW)

[3, 3, 7, 7, 7, 3, 3]


In [5]:
ROW_MARKERS = [
    SIZE_OF_ROW[x] if x == 0 else sum([SIZE_OF_ROW[y] for y in range(x+1)]) \
    for x in range(BOARD_SIZE-1)
]
print(ROW_MARKERS)

[3, 6, 13, 20, 27, 30]


In [6]:
PEG_TO_ROW = [
    len(list(filter(lambda y: y<=x,ROW_MARKERS))) for x in range(NUM_PEGS)
]
print(PEG_TO_ROW)

[0, 0, 0, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 4, 4, 4, 4, 4, 4, 4, 5, 5, 5, 6, 6, 6]


In [7]:
PEG_TO_COL = [
    x+CORNER_SIZE if x < BOARD_SIZE-2*CORNER_SIZE else \
   int(x-ROW_MARKERS[PEG_TO_ROW[x]-1]+(BOARD_SIZE-SIZE_OF_ROW[PEG_TO_ROW[x]])/2) \
   for x in range(NUM_PEGS)
]
print(PEG_TO_COL)

[2, 3, 4, 2, 3, 4, 0, 1, 2, 3, 4, 5, 6, 0, 1, 2, 3, 4, 5, 6, 0, 1, 2, 3, 4, 5, 6, 2, 3, 4, 2, 3, 4]


In [8]:
PEG_COORDINATES = [(PEG_TO_COL[x],PEG_TO_ROW[x]) for x in range(NUM_PEGS)]
print(PEG_COORDINATES)
print(PEG_COORDINATES[CENTER_PEG])


[(2, 0), (3, 0), (4, 0), (2, 1), (3, 1), (4, 1), (0, 2), (1, 2), (2, 2), (3, 2), (4, 2), (5, 2), (6, 2), (0, 3), (1, 3), (2, 3), (3, 3), (4, 3), (5, 3), (6, 3), (0, 4), (1, 4), (2, 4), (3, 4), (4, 4), (5, 4), (6, 4), (2, 5), (3, 5), (4, 5), (2, 6), (3, 6), (4, 6)]
(3, 3)


In [9]:
def valid_coordinates(p):
    if (p[0] < 0 or p[0] >= BOARD_SIZE or p[1] < 0 or p[1] >= BOARD_SIZE):
        return False
    if ((p[0] < CORNER_SIZE or p[0] >= BOARD_SIZE - CORNER_SIZE) and \
        (p[1] < CORNER_SIZE or p[1] >= BOARD_SIZE - CORNER_SIZE)):
        return False
    return True


In [10]:
def peg_list_by_coords(p):
    if not valid_coordinates(p):
        return []
    if p[1] < CORNER_SIZE:
       return [p[1]*(BOARD_SIZE-2*CORNER_SIZE)+p[0]-CORNER_SIZE]
    if p[1] >= BOARD_SIZE-CORNER_SIZE:
        return [NUM_PEGS - (BOARD_SIZE-p[1])*(BOARD_SIZE-2*CORNER_SIZE)-CORNER_SIZE+p[0]]
    return [p[0]+ROW_MARKERS[p[1]-1]]
        

In [11]:
PEG_LIST_TABLE = [
    peg_list_by_coords((j,i)) \
    for i in range(BOARD_SIZE) for j in range(BOARD_SIZE)
]
print(PEG_LIST_TABLE)


[[], [], [0], [1], [2], [], [], [], [], [3], [4], [5], [], [], [6], [7], [8], [9], [10], [11], [12], [13], [14], [15], [16], [17], [18], [19], [20], [21], [22], [23], [24], [25], [26], [], [], [27], [28], [29], [], [], [], [], [30], [31], [32], [], []]


In [12]:
def render_game_state(s):
    output = ""
    for i in range(BOARD_SIZE):
        for j in range(BOARD_SIZE):
            pegs = PEG_LIST_TABLE[i*BOARD_SIZE+j]
            if len(pegs):
                if s[pegs[0]]:
                    output += "X"
                else:
                    output += "O"
            else:
                output += " "
        output += "\n"
    print(output)
render_game_state(INITIAL_STATE)

  XXX  
  XXX  
XXXXXXX
XXXOXXX
XXXXXXX
  XXX  
  XXX  



In [13]:
def split_game_state(s):
    return [
        list(filter(lambda x: s[x],range(NUM_PEGS))),
        list(filter(lambda x: not s[x],range(NUM_PEGS)))
    ]

In [14]:
def form_pairs(a,b):
    a_len,b_len = len(a),len(b)
    return list(map(
        lambda i: [a[i%a_len],b[int((i-(i%a_len))/a_len)]],
        range(a_len*b_len)
    ))

In [15]:
def two_steps_away(jump_pair):
    if jump_pair[0][0]==jump_pair[1][0]:
        if jump_pair[0][1]==jump_pair[1][1]+2:
            return True
        if jump_pair[0][1]==jump_pair[1][1]-2:
          return True
    if jump_pair[0][1]==jump_pair[1][1]:
        if jump_pair[0][0]==jump_pair[1][0]+2:
          return True
        if jump_pair[0][0]==jump_pair[1][0]-2:
          return True
    return False

In [16]:
def jump_center(peg1,peg2):
    p1 = PEG_COORDINATES[peg1]
    p2 = PEG_COORDINATES[peg2]
    p3 = (int((p1[0]+p2[0])/2),int((p1[1]+p2[1])/2))
    return PEG_LIST_TABLE[p3[0] + p3[1]*BOARD_SIZE][0]
    

In [17]:
def valid_moves(state):
    split_state = split_game_state(state)
    possible_pairs = form_pairs(split_state[0],split_state[1])
    valid_jumps = list(filter(
        lambda x: two_steps_away([
            PEG_COORDINATES[x[0]],
            PEG_COORDINATES[x[1]]
        ]),possible_pairs))
    jump_triples = list(map(
        lambda x: [x[0],x[1],jump_center(x[0],x[1])],
        valid_jumps))
    return list(filter(lambda x: state[x[2]],jump_triples))

In [18]:
valid_moves(INITIAL_STATE)

[[4, 16, 9], [14, 16, 15], [18, 16, 17], [28, 16, 23]]

In [19]:
valid_moves(TERMINAL_STATE)

[]

In [20]:
POWERS_OF_2 = [2**i for i in range(NUM_PEGS)]
def state_to_integer(s):
    return int(np.sum(np.dot(POWERS_OF_2,s)))
    

In [21]:
state_to_integer(INITIAL_STATE)

8589869055

In [22]:
state_to_integer(TERMINAL_STATE)

65536

In [23]:
def apply_move(state,jump_triple):
    return [
        state[i] if i not in set(jump_triple) else 1-state[i] \
        for i in range(NUM_PEGS)
    ]
render_game_state(INITIAL_STATE)    
render_game_state(apply_move(INITIAL_STATE,[4, 16, 9]))

  XXX  
  XXX  
XXXXXXX
XXXOXXX
XXXXXXX
  XXX  
  XXX  

  XXX  
  XOX  
XXXOXXX
XXXXXXX
XXXXXXX
  XXX  
  XXX  



In [24]:
def possible_follow_up_states(s):
    poss_moves = valid_moves(s)
    return [
        apply_move(s,m) for m in poss_moves
    ]
print(possible_follow_up_states(INITIAL_STATE))

[[1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1]]


In [25]:
def state_from_integer(n):
    return [
        1 if n & (2**i) else 0 for i in range(NUM_PEGS)
    ]

In [26]:
render_game_state(state_from_integer(8589869055))

  XXX  
  XXX  
XXXXXXX
XXXOXXX
XXXXXXX
  XXX  
  XXX  



In [27]:
# maybe I'm thinking about this all wrong
# my "NUM_PEGS" isn't actaully the number of pegs,
# it's the number of SLOTS. 
NUM_SLOTS = NUM_PEGS
# what if I treat the pegs a little more literally
# and actually move a peg from one position to another
# removing exactly one peg from the board each time?
ACTUAL_PEG_COUNT = NUM_SLOTS - 1

In [28]:
INITIAL_PEG_SLOTS = [i if i < CENTER_PEG else i+1 for i in range(ACTUAL_PEG_COUNT)]
print(INITIAL_PEG_SLOTS)

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32]


In [29]:
PEG_DATA = [
    [s] for s in INITIAL_PEG_SLOTS
]
print(PEG_DATA)

[[0], [1], [2], [3], [4], [5], [6], [7], [8], [9], [10], [11], [12], [13], [14], [15], [17], [18], [19], [20], [21], [22], [23], [24], [25], [26], [27], [28], [29], [30], [31], [32]]


In [30]:
REVERSE_INITIAL_SLOT = {
    s:i for i,s in enumerate(INITIAL_PEG_SLOTS)
}
print(REVERSE_INITIAL_SLOT)

{0: 0, 1: 1, 2: 2, 3: 3, 4: 4, 5: 5, 6: 6, 7: 7, 8: 8, 9: 9, 10: 10, 11: 11, 12: 12, 13: 13, 14: 14, 15: 15, 17: 16, 18: 17, 19: 18, 20: 19, 21: 20, 22: 21, 23: 22, 24: 23, 25: 24, 26: 25, 27: 26, 28: 27, 29: 28, 30: 29, 31: 30, 32: 31}


In [31]:
def render_new_state(ns):
    lengths = list(map(len,ns))
    max = np.max(lengths)
    print(f"Turn: {max}")
    active_pegs = list(filter(
        lambda x: len(x)==max,
        ns))
    filled_slots = set([x[-1] for x in active_pegs])
    slot_rep = [1 if i in filled_slots else 0 for i in range(NUM_SLOTS)]
    render_game_state(slot_rep)
        
render_new_state(PEG_DATA)

Turn: 1
  XXX  
  XXX  
XXXXXXX
XXXOXXX
XXXXXXX
  XXX  
  XXX  



In [32]:
def convert_to_old_rep(ns):
    lengths = list(map(len,ns))
    max = np.max(lengths)
    active_pegs = list(filter(
        lambda x: len(x)==max,
        ns))
    filled_slots = set([x[-1] for x in active_pegs])
    return [1 if i in filled_slots else 0 for i in range(NUM_SLOTS)]


In [33]:
NEW_TERMINAL_STATE = [[CENTER_PEG]]
render_new_state(NEW_TERMINAL_STATE)

Turn: 1
  OOO  
  OOO  
OOOOOOO
OOOXOOO
OOOOOOO
  OOO  
  OOO  



In [34]:
# so, what's the big deal?
# well, this peg needs to be "one of" the original 32
PEG_COORDINATES[CENTER_PEG]

(3, 3)

In [35]:
# since pegs can only jump 2 spaces in a cardinal direction,
# only pegs originally placed in certain slots EVER access this location
def access_to_slot(s,init_data=PEG_DATA):
    a2 = PEG_COORDINATES[s][0]%2 
    b2 = PEG_COORDINATES[s][1]%2 
    
    return list(filter(
        lambda i: PEG_COORDINATES[i][0]%2 == a2 \
        and PEG_COORDINATES[i][1]%2 == b2,
        [i[0] for i in init_data]))
print(access_to_slot(16))

[4, 14, 18, 28]


In [36]:
render_game_state([1 if i in set(access_to_slot(CENTER_PEG)) else 0 for i in range(NUM_SLOTS)])

  OOO  
  OXO  
OOOOOOO
OXOOOXO
OOOOOOO
  OXO  
  OOO  



In [37]:
# basically 4 positions up to rotation
# let's choose 4 and work backwards
BACKWARDS_SOLUTIONS = [{i:16} for i in access_to_slot(16)]
print(BACKWARDS_SOLUTIONS)

[{4: 16}, {14: 16}, {18: 16}, {28: 16}]


In [38]:
access_to_slot(4)

[4, 14, 18, 28]

In [39]:
render_game_state([1 if i in set(access_to_slot(4)) else 0 for i in range(NUM_SLOTS)])

  OOO  
  OXO  
OOOOOOO
OXOOOXO
OOOOOOO
  OXO  
  OOO  



In [40]:
# backtrack a step
def undo_move(source,target,state_vec):
    assert state_vec[target]
    assert not state_vec[source]
    c = jump_center(source,target)
    assert not state_vec[c]
    flipped = set([source,target,c])
    return [
        state_vec[i] if i not in flipped else 1-state_vec[i] for i in range(NUM_SLOTS)
    ]

    

In [41]:
prevstate = undo_move(4,16,TERMINAL_STATE)
render_game_state(prevstate)

  OOO  
  OXO  
OOOXOOO
OOOOOOO
OOOOOOO
  OOO  
  OOO  



In [42]:
new_slot = jump_center(16,4)
print(new_slot)

9


In [43]:
access_to_slot(new_slot)

[1, 7, 9, 11, 21, 23, 25, 31]

In [44]:
access_to_slot(17)

[3, 5, 13, 15, 17, 19, 27, 29]

In [45]:
POSSIBLE_TERMINAL_STATES = [
    [[REVERSE_INITIAL_SLOT[x]]] for x in access_to_slot(CENTER_PEG)
]
POSSIBLE_TERMINAL_STATES

[[[4]], [[14]], [[17]], [[27]]]

In [46]:
POSSIBLE_GAME_HISTORIES = {
    0:PEG_DATA,
    -1:POSSIBLE_TERMINAL_STATES
}
print(POSSIBLE_GAME_HISTORIES)

{0: [[0], [1], [2], [3], [4], [5], [6], [7], [8], [9], [10], [11], [12], [13], [14], [15], [17], [18], [19], [20], [21], [22], [23], [24], [25], [26], [27], [28], [29], [30], [31], [32]], -1: [[[4]], [[14]], [[17]], [[27]]]}


In [47]:
def apply_jump_map(new_rep,jump):
    return [
        p+jump[p[-1]] if p[-1] in jump else p+[p[-1]]
        for p in new_rep
    ]
    
def new_possible_states(new_rep):
    s = convert_to_old_rep(new_rep)
    v = valid_moves(s)
    print(f"{new_rep} -> {s} found {v}")
    jump_maps = [
        {j[0]:[j[1]],j[2]:[]}
        for j in v
    ]
    return [apply_jump_map(new_rep,j) for j in jump_maps]
    
POSSIBLE_GAME_HISTORIES[1] = new_possible_states(PEG_DATA)
print(POSSIBLE_GAME_HISTORIES[1])

[[0], [1], [2], [3], [4], [5], [6], [7], [8], [9], [10], [11], [12], [13], [14], [15], [17], [18], [19], [20], [21], [22], [23], [24], [25], [26], [27], [28], [29], [30], [31], [32]] -> [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1] found [[4, 16, 9], [14, 16, 15], [18, 16, 17], [28, 16, 23]]
[[[0, 0], [1, 1], [2, 2], [3, 3], [4, 16], [5, 5], [6, 6], [7, 7], [8, 8], [9], [10, 10], [11, 11], [12, 12], [13, 13], [14, 14], [15, 15], [17, 17], [18, 18], [19, 19], [20, 20], [21, 21], [22, 22], [23, 23], [24, 24], [25, 25], [26, 26], [27, 27], [28, 28], [29, 29], [30, 30], [31, 31], [32, 32]], [[0, 0], [1, 1], [2, 2], [3, 3], [4, 4], [5, 5], [6, 6], [7, 7], [8, 8], [9, 9], [10, 10], [11, 11], [12, 12], [13, 13], [14, 16], [15], [17, 17], [18, 18], [19, 19], [20, 20], [21, 21], [22, 22], [23, 23], [24, 24], [25, 25], [26, 26], [27, 27], [28, 28], [29, 29], [30, 30], [31, 31], [32, 32]], [[0, 0], [1, 1], [2, 2], [3, 3], [4, 4], [5, 5], [6, 6

In [48]:
def make_removal_history(new_rep):
    lengths = []
    by_length = {}

    moved_in_turn = {}
    last_state = None
    for i,h in enumerate(new_rep):
        l = len(h)
        lengths.append(l)
        if l not in by_length:
            by_length[l]=[i]
        else:
            by_length[l].append(i)
        if l > 1:
            ival = h[0]
            for t in range(l-1):
                if h[t+1] != h[t]:
                    moved_in_turn[t] = i
                    break

    max = int(np.max(lengths))
    removal_hist = []
    if max > 1:
        for t in range(max-1):
            removed = by_length[t+1][0]
            removal_hist.append(removed)
        return [
            (r,moved_in_turn[i])
            for i,r in enumerate(removal_hist)
            
        ]            
    return []
    
        

make_removal_history(POSSIBLE_GAME_HISTORIES[1][0])

[(9, 4)]

In [49]:
for o in POSSIBLE_GAME_HISTORIES[1]:
    render_new_state(o)
    print(make_removal_history(o))

Turn: 2
  XXX  
  XOX  
XXXOXXX
XXXXXXX
XXXXXXX
  XXX  
  XXX  

[(9, 4)]
Turn: 2
  XXX  
  XXX  
XXXXXXX
XOOXXXX
XXXXXXX
  XXX  
  XXX  

[(15, 14)]
Turn: 2
  XXX  
  XXX  
XXXXXXX
XXXXOOX
XXXXXXX
  XXX  
  XXX  

[(16, 17)]
Turn: 2
  XXX  
  XXX  
XXXXXXX
XXXXXXX
XXXOXXX
  XOX  
  XXX  

[(22, 27)]


In [50]:
grouping_render = ""
slot_groups = {i:[] for i in range(4)}
for i in range(BOARD_SIZE):
    i_2 = i%2
    for j in range(BOARD_SIZE):
        j_2 = j%2
        pegs = PEG_LIST_TABLE[i*BOARD_SIZE+j]
        if len(pegs):
            color = j_2*2+i_2
            grouping_render += str(color)
            slot_groups[color].append(pegs[0])
        else:
            grouping_render += " "
    grouping_render += "\n"
print(grouping_render)
print(slot_groups)
    

  020  
  131  
0202020
1313131
0202020
  131  
  020  

{0: [0, 2, 6, 8, 10, 12, 20, 22, 24, 26, 30, 32], 1: [3, 5, 13, 15, 17, 19, 27, 29], 2: [1, 7, 9, 11, 21, 23, 25, 31], 3: [4, 14, 16, 18, 28]}


In [51]:
for i in range(4):
    print(len(slot_groups[i]))
print(sum([len(slot_groups[i]) for i in range(4)]))

12
8
8
5
33


In [52]:
REVERSE_INITIAL_SLOT[32]

31

In [53]:
peg_groups = {}
for i in range(4):
    cur_group = slot_groups[i]
    if i == 3:
        cur_group = list(filter(lambda x: x!=16,cur_group))
    peg_groups[i] = list(map(lambda x: REVERSE_INITIAL_SLOT[x],cur_group))
print(peg_groups)
    
        

{0: [0, 2, 6, 8, 10, 12, 19, 21, 23, 25, 29, 31], 1: [3, 5, 13, 15, 16, 18, 26, 28], 2: [1, 7, 9, 11, 20, 22, 24, 30], 3: [4, 14, 17, 27]}


In [54]:
print(len(peg_groups[0])+len(peg_groups[3]))

16


In [55]:
print(len(peg_groups[1])+len(peg_groups[2]))

16


In [56]:
ADJACENT_GROUPS = {
    0:[1,2],
    1:[0,3],
    2:[0,3],
    3:[1,2]
}
POSSIBLE_REMOVAL_PAIRS = []
for i in range(4):
    cur_group = peg_groups[i]
    combined_adjs = peg_groups[ADJACENT_GROUPS[i][0]]+peg_groups[ADJACENT_GROUPS[i][1]]
    for j in cur_group:
        for k in combined_adjs:
            POSSIBLE_REMOVAL_PAIRS.append((j,k))
print(POSSIBLE_REMOVAL_PAIRS)

[(0, 3), (0, 5), (0, 13), (0, 15), (0, 16), (0, 18), (0, 26), (0, 28), (0, 1), (0, 7), (0, 9), (0, 11), (0, 20), (0, 22), (0, 24), (0, 30), (2, 3), (2, 5), (2, 13), (2, 15), (2, 16), (2, 18), (2, 26), (2, 28), (2, 1), (2, 7), (2, 9), (2, 11), (2, 20), (2, 22), (2, 24), (2, 30), (6, 3), (6, 5), (6, 13), (6, 15), (6, 16), (6, 18), (6, 26), (6, 28), (6, 1), (6, 7), (6, 9), (6, 11), (6, 20), (6, 22), (6, 24), (6, 30), (8, 3), (8, 5), (8, 13), (8, 15), (8, 16), (8, 18), (8, 26), (8, 28), (8, 1), (8, 7), (8, 9), (8, 11), (8, 20), (8, 22), (8, 24), (8, 30), (10, 3), (10, 5), (10, 13), (10, 15), (10, 16), (10, 18), (10, 26), (10, 28), (10, 1), (10, 7), (10, 9), (10, 11), (10, 20), (10, 22), (10, 24), (10, 30), (12, 3), (12, 5), (12, 13), (12, 15), (12, 16), (12, 18), (12, 26), (12, 28), (12, 1), (12, 7), (12, 9), (12, 11), (12, 20), (12, 22), (12, 24), (12, 30), (19, 3), (19, 5), (19, 13), (19, 15), (19, 16), (19, 18), (19, 26), (19, 28), (19, 1), (19, 7), (19, 9), (19, 11), (19, 20), (19, 22)

In [57]:
def group_by_slot(s):
    for i in range(4):
        if s in set(slot_groups[i]):
            return i
    assert False
    
def group_by_peg(p):
    for i in range(4):
        if p in set(peg_groups[i]):
            return i
    assert False
    

In [58]:
def filter_by_left_value(rp,v):
    return list(filter(lambda x: x[0]==v,rp))
print(filter_by_left_value(POSSIBLE_REMOVAL_PAIRS,4))
print(filter_by_left_value(POSSIBLE_REMOVAL_PAIRS,9))
def filter_by_right_value(rp,v):
    return list(filter(lambda x: x[1]==v,rp))
print(filter_by_right_value(POSSIBLE_REMOVAL_PAIRS,4))
print(filter_by_right_value(POSSIBLE_REMOVAL_PAIRS,9))
print(filter_by_right_value(POSSIBLE_REMOVAL_PAIRS,17))

[(4, 3), (4, 5), (4, 13), (4, 15), (4, 16), (4, 18), (4, 26), (4, 28), (4, 1), (4, 7), (4, 9), (4, 11), (4, 20), (4, 22), (4, 24), (4, 30)]
[(9, 0), (9, 2), (9, 6), (9, 8), (9, 10), (9, 12), (9, 19), (9, 21), (9, 23), (9, 25), (9, 29), (9, 31), (9, 4), (9, 14), (9, 17), (9, 27)]
[(3, 4), (5, 4), (13, 4), (15, 4), (16, 4), (18, 4), (26, 4), (28, 4), (1, 4), (7, 4), (9, 4), (11, 4), (20, 4), (22, 4), (24, 4), (30, 4)]
[(0, 9), (2, 9), (6, 9), (8, 9), (10, 9), (12, 9), (19, 9), (21, 9), (23, 9), (25, 9), (29, 9), (31, 9), (4, 9), (14, 9), (17, 9), (27, 9)]
[(3, 17), (5, 17), (13, 17), (15, 17), (16, 17), (18, 17), (26, 17), (28, 17), (1, 17), (7, 17), (9, 17), (11, 17), (20, 17), (22, 17), (24, 17), (30, 17)]


In [59]:
def filter_ne_left_value(rp,v):
    return list(filter(lambda x: x[0]!=v,rp))
print(filter_ne_left_value(POSSIBLE_REMOVAL_PAIRS,4))
def filter_ne_right_value(rp,v):
    return list(filter(lambda x: x[1]!=v,rp))
print(filter_ne_right_value(POSSIBLE_REMOVAL_PAIRS,4))


[(0, 3), (0, 5), (0, 13), (0, 15), (0, 16), (0, 18), (0, 26), (0, 28), (0, 1), (0, 7), (0, 9), (0, 11), (0, 20), (0, 22), (0, 24), (0, 30), (2, 3), (2, 5), (2, 13), (2, 15), (2, 16), (2, 18), (2, 26), (2, 28), (2, 1), (2, 7), (2, 9), (2, 11), (2, 20), (2, 22), (2, 24), (2, 30), (6, 3), (6, 5), (6, 13), (6, 15), (6, 16), (6, 18), (6, 26), (6, 28), (6, 1), (6, 7), (6, 9), (6, 11), (6, 20), (6, 22), (6, 24), (6, 30), (8, 3), (8, 5), (8, 13), (8, 15), (8, 16), (8, 18), (8, 26), (8, 28), (8, 1), (8, 7), (8, 9), (8, 11), (8, 20), (8, 22), (8, 24), (8, 30), (10, 3), (10, 5), (10, 13), (10, 15), (10, 16), (10, 18), (10, 26), (10, 28), (10, 1), (10, 7), (10, 9), (10, 11), (10, 20), (10, 22), (10, 24), (10, 30), (12, 3), (12, 5), (12, 13), (12, 15), (12, 16), (12, 18), (12, 26), (12, 28), (12, 1), (12, 7), (12, 9), (12, 11), (12, 20), (12, 22), (12, 24), (12, 30), (19, 3), (19, 5), (19, 13), (19, 15), (19, 16), (19, 18), (19, 26), (19, 28), (19, 1), (19, 7), (19, 9), (19, 11), (19, 20), (19, 22)

In [60]:
len(POSSIBLE_REMOVAL_PAIRS)

512

In [61]:
rm_hist_start = list(map(make_removal_history,POSSIBLE_GAME_HISTORIES[1]))
print(rm_hist_start)

[[(9, 4)], [(15, 14)], [(16, 17)], [(22, 27)]]


In [62]:
rm_hist_end = [filter_by_right_value(POSSIBLE_REMOVAL_PAIRS,ts[0][0]) for ts in POSSIBLE_GAME_HISTORIES[-1]]
print(rm_hist_end)

[[(3, 4), (5, 4), (13, 4), (15, 4), (16, 4), (18, 4), (26, 4), (28, 4), (1, 4), (7, 4), (9, 4), (11, 4), (20, 4), (22, 4), (24, 4), (30, 4)], [(3, 14), (5, 14), (13, 14), (15, 14), (16, 14), (18, 14), (26, 14), (28, 14), (1, 14), (7, 14), (9, 14), (11, 14), (20, 14), (22, 14), (24, 14), (30, 14)], [(3, 17), (5, 17), (13, 17), (15, 17), (16, 17), (18, 17), (26, 17), (28, 17), (1, 17), (7, 17), (9, 17), (11, 17), (20, 17), (22, 17), (24, 17), (30, 17)], [(3, 27), (5, 27), (13, 27), (15, 27), (16, 27), (18, 27), (26, 27), (28, 27), (1, 27), (7, 27), (9, 27), (11, 27), (20, 27), (22, 27), (24, 27), (30, 27)]]


In [63]:
intro_outro_pairs = []
for s in rm_hist_start:
    for t in rm_hist_end:
        for p in t:
            intro_outro_pairs.append((s,[p]))
print(intro_outro_pairs)

[([(9, 4)], [(3, 4)]), ([(9, 4)], [(5, 4)]), ([(9, 4)], [(13, 4)]), ([(9, 4)], [(15, 4)]), ([(9, 4)], [(16, 4)]), ([(9, 4)], [(18, 4)]), ([(9, 4)], [(26, 4)]), ([(9, 4)], [(28, 4)]), ([(9, 4)], [(1, 4)]), ([(9, 4)], [(7, 4)]), ([(9, 4)], [(9, 4)]), ([(9, 4)], [(11, 4)]), ([(9, 4)], [(20, 4)]), ([(9, 4)], [(22, 4)]), ([(9, 4)], [(24, 4)]), ([(9, 4)], [(30, 4)]), ([(9, 4)], [(3, 14)]), ([(9, 4)], [(5, 14)]), ([(9, 4)], [(13, 14)]), ([(9, 4)], [(15, 14)]), ([(9, 4)], [(16, 14)]), ([(9, 4)], [(18, 14)]), ([(9, 4)], [(26, 14)]), ([(9, 4)], [(28, 14)]), ([(9, 4)], [(1, 14)]), ([(9, 4)], [(7, 14)]), ([(9, 4)], [(9, 14)]), ([(9, 4)], [(11, 14)]), ([(9, 4)], [(20, 14)]), ([(9, 4)], [(22, 14)]), ([(9, 4)], [(24, 14)]), ([(9, 4)], [(30, 14)]), ([(9, 4)], [(3, 17)]), ([(9, 4)], [(5, 17)]), ([(9, 4)], [(13, 17)]), ([(9, 4)], [(15, 17)]), ([(9, 4)], [(16, 17)]), ([(9, 4)], [(18, 17)]), ([(9, 4)], [(26, 17)]), ([(9, 4)], [(28, 17)]), ([(9, 4)], [(1, 17)]), ([(9, 4)], [(7, 17)]), ([(9, 4)], [(9, 17)])

In [64]:
len(intro_outro_pairs)

256

In [65]:
intro_outro_pairs[0]

([(9, 4)], [(3, 4)])

In [66]:
def forward_state_regen(removal_sequence):
    peg_state = [[x] if x < CENTER_PEG else [x+1] for x in range(32)]
    removed_pegs = set([])
    open_slots = set({CENTER_PEG})
    num_moves = len(removal_sequence)
    print(f"init:{peg_state}\nrem seq:{removal_sequence}\nL:{num_moves}")

    for r in removal_sequence:
        board_state = convert_to_old_rep(peg_state)
        v_moves = valid_moves(board_state)
        # note that we're matching up different domains here (pegs/slots)
        matching_moves = list(filter(
            lambda m: m[2]==peg_state[r[0]][-1] and m[0]==peg_state[r[1]][-1],
            v_moves))
        assert len(matching_moves)==1
        move = matching_moves[0]
        assert move[1] in open_slots
        # the peg that does the jumping moves to the open slot
        orig_slot =peg_state[r[1]][-1]
        peg_state[r[1]].append(move[1])
        open_slots.add(orig_slot)
        open_slots.remove(move[1])
        removed_pegs.add(r[0])
        # all pegs except jump pair stay put for another turn
        for i,e in enumerate(peg_state):
            if i != r[0] and i != r[1]:
                peg_state[i].append(e[-1])
        print(f"new peg state {peg_state}")
    return peg_state,open_slots,removed_pegs


In [67]:
forward_state_regen(intro_outro_pairs[0][0])

init:[[0], [1], [2], [3], [4], [5], [6], [7], [8], [9], [10], [11], [12], [13], [14], [15], [17], [18], [19], [20], [21], [22], [23], [24], [25], [26], [27], [28], [29], [30], [31], [32]]
rem seq:[(9, 4)]
L:1
new peg state [[0, 0], [1, 1], [2, 2], [3, 3], [4, 16], [5, 5], [6, 6], [7, 7], [8, 8], [9], [10, 10], [11, 11], [12, 12], [13, 13], [14, 14], [15, 15], [17, 17], [18, 18], [19, 19], [20, 20], [21, 21], [22, 22], [23, 23], [24, 24], [25, 25], [26, 26], [27, 27], [28, 28], [29, 29], [30, 30], [31, 31], [32, 32]]


([[0, 0],
  [1, 1],
  [2, 2],
  [3, 3],
  [4, 16],
  [5, 5],
  [6, 6],
  [7, 7],
  [8, 8],
  [9],
  [10, 10],
  [11, 11],
  [12, 12],
  [13, 13],
  [14, 14],
  [15, 15],
  [17, 17],
  [18, 18],
  [19, 19],
  [20, 20],
  [21, 21],
  [22, 22],
  [23, 23],
  [24, 24],
  [25, 25],
  [26, 26],
  [27, 27],
  [28, 28],
  [29, 29],
  [30, 30],
  [31, 31],
  [32, 32]],
 {4},
 {9})

In [68]:
def adjacent_slots(s):
    coords = PEG_COORDINATES[s]
    above = (coords[0],coords[1]-1)
    below = (coords[0],coords[1]+1)
    right = (coords[0]+1,coords[1])
    left = (coords[0]-1,coords[1])
    output = []
    for p in [above,below,left,right]:
        output.extend(peg_list_by_coords(p))
    return output
adjacent_slots(16)
    

[9, 23, 15, 17]

In [69]:
def empty_adjacent_slots(filled_slot_set):
    adj_list = list(map(adjacent_slots,filled_slot_set))
    return set(list(reduce(lambda x,y: x+y,adj_list)))

empty_adjacent_slots(set([16,17]))

{9, 10, 15, 16, 17, 18, 23, 24}

In [70]:
def slots_to_possible_peg_groups(slot_list):
    peg_list_by_slot = {}
    for s in slot_list:
        g = group_by_slot(s)
        peg_list_by_slot[s] = g
    return peg_list_by_slot
slots_to_possible_peg_groups([16])    

{16: 3}

In [71]:
def jump_origin(destination,center):
    p1 = PEG_COORDINATES[destination]
    p2 = PEG_COORDINATES[center]
    p3 = ((p2[0]*2-p1[0]),(p2[1]*2-p1[1]))
    return PEG_LIST_TABLE[p3[0] + p3[1]*BOARD_SIZE][0]
jump_origin(16,15)

14

In [72]:
def backwards_state_regen(removal_sequence):
    num_moves = len(removal_sequence)
    assert num_moves > 0
    print(removal_sequence)
    active_pegs = set([removal_sequence[0][1]])
    filled_slots = set([CENTER_PEG])
    slot_rep = [1 if i == CENTER_PEG else 0 for i in range(NUM_SLOTS)]
    render_game_state(slot_rep)
    peg_rep = [[] if i != removal_sequence[0][1] else [CENTER_PEG] for i in range(ACTUAL_PEG_COUNT)]
    print(peg_rep)
    empty_slots = empty_adjacent_slots(filled_slots)
    print(empty_slots)
    slots_to_groups = slots_to_possible_peg_groups(empty_slots)
    print(slots_to_groups)
    origin_group = group_by_peg(removal_sequence[0][0])
    print(origin_group)
    empty_slots_from_group = []
    for k,v in zip(slots_to_groups.keys(),slots_to_groups.values()):
        if v == origin_group:
            empty_slots_from_group.append(k)
    print(empty_slots_from_group)

    possible_previous_states = []
    for s in empty_slots_from_group:
        print(f"Attempting to fill slot {s} with peg {removal_sequence[0][0]}")
        jo = jump_origin(CENTER_PEG,s)
        
        d_state = [
            [] if x not in set(removal_sequence[0]) else \
            [s] if x == removal_sequence[0][0] else [jo] \
            for x in range(ACTUAL_PEG_COUNT)
        ]
        new_state = [i+j for i,j in zip(d_state,peg_rep)]
        possible_previous_states.append(new_state)
    print(f"Found {len(possible_previous_states)} previous states")
    return possible_previous_states


backwards_state_regen(intro_outro_pairs[0][1])

[(3, 4)]
  OOO  
  OOO  
OOOOOOO
OOOXOOO
OOOOOOO
  OOO  
  OOO  

[[], [], [], [], [16], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], []]
{9, 15, 17, 23}
{9: 2, 15: 1, 17: 1, 23: 2}
1
[15, 17]
Attempting to fill slot 15 with peg 3
Attempting to fill slot 17 with peg 3
Found 2 previous states


[[[],
  [],
  [],
  [15],
  [14, 16],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  []],
 [[],
  [],
  [],
  [17],
  [18, 16],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  []]]

In [75]:
reduce(lambda x,y: x*y,map(lambda i: i+1,range(29)))

8841761993739701954543616000000